# CHAPTER 39 - DECORATORS

## THE BASICS 

In [4]:
def decorator(cls):
    class Wraper:
        def __init__(self, *args):
            self.wrapper = cls(*args)
        def __getattr__(self, name):
            return getattr(self.wrapper, name)
    return Wraper

In [5]:
@decorator
class C:
    def __init__(self, x, y):
        self.attr = 'spam'

In [6]:
x = C(6,7)

In [8]:
x.attr

'spam'

## CODING FUNCTIONS DECORATORS

In [10]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args):
        self.calls += 1
        print(f'calls {self.calls} to {self.func.__name__}')

In [11]:
@tracer
def spam(a, b, c):
    print(a + b + C)

In [12]:
spam(1, 2, 3)

calls 1 to spam


In [13]:
spam('a', 'b', 'c')

calls 2 to spam


In [14]:
spam.calls

2

## DECORATOR STATE RETENTION OPTIONS